In [1]:
import os
from utils import *
import optuna

In [2]:
data, usermap, itemmap, users = load_data2()
usermap_reverse = {v: k for k, v in usermap.items()}
itemmap_reverse = {v: k for k, v in itemmap.items()}
data_train, data_test, data_val = split_data2(data, 0, 0.2)

In [3]:
from Recommenders.BiVAE.dataset import Dataset

data_train = Dataset.build(data_train, global_uid_map=usermap, global_iid_map=itemmap)
data_val = Dataset.build(data_val, global_uid_map=usermap, global_iid_map=itemmap)

In [4]:
def evaluator(recommender: object, data_train, data_test: sp.csr_matrix, recommendation_length: int = 10):
    accum_precision = 0
    accum_recall = 0
    accum_map = 0

    num_users = len(data_train.user_ids)

    num_users_evaluated = 0
    num_users_skipped = 0
    for user_id in range(num_users):
        user_profile_start = data_test.indptr[user_id]
        user_profile_end = data_test.indptr[user_id+1]

        relevant_items = data_test.indices[user_profile_start:user_profile_end]

        if relevant_items.size == 0:
            num_users_skipped += 1
            continue
        
        if not usermap_reverse.get(user_id):
            num_users_skipped += 1
            continue

        recommendations = np.array(recommender.recommend(usermap_reverse[user_id], k=recommendation_length))

        accum_precision += precision(recommendations, relevant_items)
        accum_recall += recall(recommendations, relevant_items)
        accum_map += mean_average_precision(recommendations, relevant_items)

        num_users_evaluated += 1


    accum_precision /= max(num_users_evaluated, 1)
    accum_recall /= max(num_users_evaluated, 1)
    accum_map /=  max(num_users_evaluated, 1)

    return accum_precision, accum_recall, accum_map, num_users_evaluated, num_users_skipped

In [5]:
study_name = "BiVAE"
study = optuna.create_study(
    study_name=study_name,
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

[I 2023-12-18 22:03:00,788] A new study created in RDB with name: BiVAE


In [6]:
from Recommenders.BiVAE.BiVAECF import BiVAECF

def objective(trial):
    encoder_structure = trial.suggest_categorical('encoder_structure', [20, 40, 60, 80, 100, 200, 300])
    act_fn = trial.suggest_categorical('act_fn', ['sigmoid', 'tanh', 'elu', 'relu', 'relu6'])
    likelihood = trial.suggest_categorical('likelihood', ['pois', 'gaus', 'bern'])
    epochs = trial.suggest_int('epochs', 10, 1000)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256, 512, 1024, 2048, 4096])
    learning_rate = trial.suggest_float('learning_rate', 1e-6, 1e-1, log=True)

    recommender = BiVAECF(
        k=10,
        encoder_structure=[encoder_structure],
        act_fn=act_fn,
        likelihood=likelihood,
        n_epochs=epochs,
        batch_size=batch_size,
        learning_rate=learning_rate,
        verbose=True,
        use_gpu=True,
    )
    recommender.fit(data_train)
    _, _, ev_map, _, _ = evaluator(recommender, data_train, data_val.matrix)
    
    return ev_map

study.optimize(objective, n_trials=150)


100%|██████████| 338/338 [24:01<00:00,  4.27s/it, loss_i=2.89, loss_u=3.78]
[I 2023-12-18 22:27:43,998] Trial 0 finished with value: 0.0001265981756139996 and parameters: {'encoder_structure': 100, 'act_fn': 'tanh', 'likelihood': 'bern', 'epochs': 338, 'batch_size': 512, 'learning_rate': 2.974327046123427e-06}. Best is trial 0 with value: 0.0001265981756139996.
  8%|▊         | 28/357 [02:01<23:42,  4.32s/it, loss_i=0.885, loss_u=0.624]
[W 2023-12-18 22:29:58,426] Trial 1 failed with parameters: {'encoder_structure': 300, 'act_fn': 'elu', 'likelihood': 'gaus', 'epochs': 357, 'batch_size': 256, 'learning_rate': 3.0752343901796736e-05} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/kinami/.conda/envs/RecSysFramework/lib/python3.8/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_5015/2187087351.py", line 22, in objective
    recommender.fit(data_train)
  File "/

KeyboardInterrupt: 